In [2]:
import sys

from evoman.environment import Environment
from demo_controller import player_controller

# imports other libs
import time
import numpy as np
from math import fabs,sqrt
import glob, os

pygame 2.5.2 (SDL 2.28.3, Python 3.11.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [15]:
n_vars = 10
dom_u = 1
dom_l = -1
npop = 100
gens = 30
mutation = 0.2
last_best = 0



In [107]:
def crossover(pop, fixed_start=True, fixed_end=True, n_offspring=2, p_left=0.5, p_mutation = 0.2):
    
    total_offspring = np.zeros((0,n_vars))
    
    for p in range(0, pop.shape[0], 2):  # stepsize 2, since you choose 2 parents and otherwise you get 2 times the number of offspring
        parents = np.zeros((2, pop.shape[1]))
        parents[0] = parent_selection(pop)
        parents[1] = parent_selection(pop)
        
        offspring = np.zeros((n_offspring, n_vars))
        
        index_list = np.arange(0, len(parents[0])).tolist() #create a list of index to sample from
    
        if fixed_start:
            index_list.remove(0) #Remove the first index 0 from the list
    
        if fixed_end:
            index_list.remove(len(parents[0])-1) #Remove the last index from the list
        
        #Sample one integers from the index list
        a = np.random.choice(index_list)
             
        for c in range(len(offspring)):
            if np.random.uniform() <= p_left:  # recombine left part
                offspring[c, a:] = parents[c, a:]
                offspring[c, :a] = (parents[0, :a] + parents[1, :a]) / 2

            else:  # recombine right part
                offspring[c, :a] = parents[c, :a]
                offspring[c, a:] = (parents[c, a:] + parents[1, a:]) / 2
                
            # mutation 
            for i in range(len(offspring[c])):
                if np.random.uniform(0,1) <= p_mutation:
                    offspring[c, i] = np.random.uniform(-1, 1)
            
            total_offspring = np.vstack((total_offspring, offspring[c]))
     
    return total_offspring
        

In [100]:
pop = np.random.uniform(dom_l, dom_u, (10, n_vars))
fit_pop = np.linspace(1, 10, 10)

In [106]:
def parent_selection(pop, iNum_of_parents=6):
    
    # Select n random parents ids
    random_parents_id = np.random.choice(pop.shape[0], iNum_of_parents, replace=False)
    
    random_parents_fitness = fit_pop[random_parents_id]
    
    best_parent = np.where(random_parents_fitness == np.max(random_parents_fitness))


    # Select the n according individuals
    # random_parents_fit = np.array(fit_pop[i] for i in len(random_parents_id))
    
    return pop[best_parent]

    # # Find the index of the parent with the highest fitness score
    # best_parent_id = np.argmax(random_parents[:,1])
    # parent_one = random_parents[best_parent_id,0]

    # # Create a new matrix without the best parent
    # random_parents = np.delete(random_parents, best_parent_id, axis=0)

    # # select the second parent
    # second_best = np.argmax(random_parents[:,1])
    # second_parent = random_parents[second_best,0]

    # # Select the row with the maximum value in the first column
    # return int(parent_one), int(second_parent)

In [103]:
parent_selection(pop)

((array([1]),),
 array([[ 0.26068031,  0.76290502,  0.94647049, -0.25614783,  0.54766479,
          0.13706722,  0.14967681, -0.65007128,  0.23176116,  0.63674793]]))

In [104]:
pop[9]

array([ 0.71264934,  0.27081956,  0.0408217 ,  0.97916637,  0.56554744,
       -0.56685816,  0.06658583,  0.62295572,  0.02617009,  0.64810607])

In [105]:
pop

array([[-0.36668572, -0.72445688,  0.90914999,  0.24322528, -0.78926402,
        -0.15787338,  0.8668384 , -0.49667703,  0.21914836,  0.63725011],
       [ 0.26068031,  0.76290502,  0.94647049, -0.25614783,  0.54766479,
         0.13706722,  0.14967681, -0.65007128,  0.23176116,  0.63674793],
       [ 0.25966583,  0.48374922,  0.73704626,  0.56324543, -0.667511  ,
         0.02865414, -0.26818811,  0.8281812 , -0.05623818, -0.64921138],
       [-0.9019194 , -0.69276731, -0.94726995, -0.49092537,  0.88741809,
         0.23810801, -0.2156478 , -0.69771512,  0.33258455,  0.57487299],
       [ 0.23376349,  0.1102021 , -0.21626458,  0.81018701, -0.30258484,
        -0.25904084, -0.84049876,  0.73607359,  0.92186809, -0.30060091],
       [-0.07881701,  0.05004033,  0.42328278, -0.74315574, -0.18002084,
        -0.85121094, -0.83339445,  0.04898984, -0.8507073 , -0.47663747],
       [ 0.60948256, -0.76768601,  0.68571786, -0.73214201,  0.95226864,
         0.16541555, -0.57622683, -0.90777711

In [ ]:
def parent_selection(pop, iNum_of_parents=6):
    
    # Select n random parents ids
    random_parents_id = np.random.choice(pop.shape[0], iNum_of_parents, replace=False)
    
    random_parents_fitness = fit_pop[random_parents_id]
    
    best_parent = np.where(random_parents_fitness == np.max(random_parents_fitness))

    return pop[best_parent][0]


# limits
def limits(x):

    if x>dom_u:
        return dom_u
    elif x<dom_l:
        return dom_l
    else:
        return x


# crossover
def crossover(pop, fixed_start=True, fixed_end=True, n_vars=None, n_offspring=2, p_left=0.5, p_mutation = 0.2):
    
    total_offspring = np.zeros((0,n_vars))
    
    for p in range(0, pop.shape[0], 2):  # stepsize 2, since you choose 2 parents and otherwise you get 2 times the number of offspring
        parents = np.zeros((2, pop.shape[1]))
        parents[0] = parent_selection(pop)
        parents[1] = parent_selection(pop)
        
        offspring = np.zeros((n_offspring, n_vars))
        
        index_list = np.arange(0, len(parents[0])).tolist() #create a list of index to sample from
    
        if fixed_start:
            index_list.remove(0) #Remove the first index 0 from the list
    
        if fixed_end:
            index_list.remove(len(parents[0])-1) #Remove the last index from the list
        
        #Sample one integers from the index list
        a = np.random.choice(index_list)
             
        for c in range(len(offspring)):
            if np.random.uniform() <= p_left:  # recombine left part
                offspring[c, a:] = parents[c, a:]
                offspring[c, :a] = (parents[0, :a] + parents[1, :a]) / 2

            else:  # recombine right part
                offspring[c, :a] = parents[c, :a]
                offspring[c, a:] = (parents[c, a:] + parents[1, a:]) / 2
                
            # mutation 
            for i in range(len(offspring[c])):
                if np.random.uniform(0,1) <= p_mutation:
                    offspring[c, i] = np.random.uniform(-1, 1)
            
            total_offspring = np.vstack((total_offspring, offspring[c]))
     
    return total_offspring
